In [50]:
using ModelingToolkit, OrdinaryDiffEq, Clapeyron
using ModelingToolkit: t_nounits as t
using CoolProp.CoolProp: PropsSI

In [51]:
## Clapeyron not working

# Helper functions
function calc_ρ(T, p, fluid_model)
    return mass_density(fluid_model, p, T)
end

function create_fluid_model(fluid_name)
    return SingleFluid(fluid_name)
end

@register_symbolic calc_ρ(T::Real, p::Real, fluid_model)
@register_symbolic create_fluid_model(fluid_name::String)

# Fluid parameters
fluid_name = "Hydrogen"
T_in = 70               # K
p_in = 350e5           # Pa

# MTK model
@mtkmodel DensityCalc begin

    @parameters begin
        T
        p
        fluid_name::String
    end

    @variables begin
        ρ(t)
    end

    begin
        fluid_model = create_fluid_model(fluid_name)
    end

    @equations begin
        ρ ~ calc_ρ(T, p, fluid_model)
    end
end;

In [52]:
@named sys_ = DensityCalc(; T=T_in, p=p_in, fluid_name=fluid_name)

Model sys_:
Equations (1):
  1 standard: see equations(sys_)
Unknowns (1): see unknowns(sys_)
  ρ(t)
Parameters (3): see parameters(sys_)
  T [defaults to 70]
  p [defaults to 3.5e7]
  fluid_name [defaults to "Hydrogen"]

In [53]:
sys = complete(sys_)

Model sys_:
Equations (1):
  1 standard: see equations(sys_)
Unknowns (1): see unknowns(sys_)
  ρ(t)
Parameters (3): see parameters(sys_)
  p [defaults to 3.5e7]
  T [defaults to 70]
  fluid_name [defaults to "Hydrogen"]

In [54]:
sys = structural_simplify(sys_)

Model sys_:
Parameters (3): see parameters(sys_)
  p [defaults to 3.5e7]
  T [defaults to 70]
  fluid_name [defaults to "Hydrogen"]
Observed (1): see observed(sys_)

In [55]:
prob = ODEProblem(sys, [], (0, 10.0))
sol = solve(prob, Tsit5())
println(sol[sys.ρ][end])

67.30013202474127
